In [3]:
from Game import *
import pandas as pd

%load_ext autoreload
%autoreload 2

In [4]:
week = Game('FanDuel-NFL-2022 ET-12 ET-11 ET-84373-players-list.csv',
            'NFL',
            '2022-14 Fantasy Pros Projections 22-12-10.xlsx',
            'FantasyPros NFL Historical.xlsx',
            2022, 14)

In [5]:
startem = pd.read_excel('Sports Illustrated.xlsx')
startem.shape

(54, 2)

In [6]:
# drop all players not in SI start em
DROP = set(week.data['Nickname']) - set(startem['Name'])

week.generateRosters('reimagined', drop_players_initial=DROP, min_pts=0, drop_injuries=['D', 'IA', 'O', 'IR'], max_te=1)

Preprocessing completed in 0:00:00.003000.

Beginning strategy: reimagined
--------------------------------------------------

Initial iteration completed in 0:00:24.890969.
Optimal solutions: 511 / 511

Dropped Justin Jefferson. Using Roster1 as basis...
Iteration 1 completed in 0:00:22.919535.
Optimal solutions: 511 / 511

Dropped Tony Pollard. Using Roster2 as basis...
Iteration 2 completed in 0:00:24.769036.
Optimal solutions: 511 / 511

Dropped D'Andre Swift. Using Roster3 as basis...
Iteration 3 completed in 0:00:24.442918.
Optimal solutions: 511 / 511

Dropped Jared Goff. Using Roster4 as basis...
Iteration 4 completed in 0:00:24.852781.
Optimal solutions: 511 / 511

Dropped DJ Moore. Using Roster5 as basis...
Iteration 5 completed in 0:00:25.032524.
Optimal solutions: 511 / 511

Dropped Jerry Jeudy. Using Roster6 as basis...
Iteration 6 completed in 0:00:25.503914.
Optimal solutions: 511 / 511

Dropped Zonovan Knight. Using Roster7 as basis...
Iteration 7 completed in 0:00:25.1

(5111, 5111)

In [7]:
# verify dropped players
week.tracking['preprocessing_drops'].sort_values('Projected', ascending=False).head(10)

,Nickname,Position,Salary,Injury Indicator,QB,RB,WR,TE,DST,FLEX,Projected,MR_Period,MR_Season,MR_WeekNum,N_Games,Hist_Avg,Hist_Std,Proj_Percent
24,Geno Smith,QB,7700,NaN,1,0,0,0,0,0,19.0,2022-13,2022.0,13.0,16.0,17.90,6.44,82.6
43,Deshaun Watson,QB,7000,NaN,1,0,0,0,0,0,16.8,2022-13,2022.0,13.0,16.0,22.56,7.23,78.0
37,Tom Brady,QB,7100,NaN,1,0,0,0,0,0,16.3,2022-13,2022.0,13.0,16.0,16.50,5.87,87.8
45,Daniel Jones,QB,6900,NaN,1,0,0,0,0,0,16.3,2022-13,2022.0,13.0,16.0,16.04,5.77,57.5
50,Mike White,QB,6800,NaN,1,0,0,0,0,0,15.6,2022-13,2022.0,13.0,7.0,13.76,10.48,15.9
58,Kenny Pickett,QB,6600,NaN,1,0,0,0,0,0,15.4,2022-13,2022.0,13.0,9.0,13.21,3.47,11.5
53,Ryan Tannehill,QB,6700,NaN,1,0,0,0,0,0,15.4,2022-13,2022.0,13.0,16.0,14.29,6.21,13.4
65,Russell Wilson,QB,6500,NaN,1,0,0,0,0,0,14.6,2022-13,2022.0,13.0,16.0,15.35,6.82,47.9
66,Brock Purdy,QB,6500,NaN,1,0,0,0,0,0,14.4,2022-13,2022.0,13.0,4.0,4.30,7.37,0.6
56,Sam Darnold,QB,6600,NaN,1,0,0,0,0,0,13.9,2022-12,2022.0,12.0,16.0,14.46,8.02,1.3


In [8]:
top = week.selectRosters(.75)
week._selection

'Roster0'

In [9]:
# players with appearances
x = week.data[['Nickname', 'Position', 'Salary', 'Projected', 'Top Appearances']]
x[x['Top Appearances'] > 0].sort_values('Top Appearances', ascending=False)

,Nickname,Position,Salary,Projected,Top Appearances
27,Tony Pollard,RB,7500,15.1,1278
63,Jerry Jeudy,WR,6500,10.7,1103
34,D'Andre Swift,RB,7200,13.2,1028
0,Justin Jefferson,WR,9300,21.5,1023
175,Greg Dulcich,TE,5400,8.9,1001
38,Jared Goff,QB,7100,18.3,959
658,Pittsburgh Steelers,D,3800,7.5,879
67,Adam Thielen,WR,6400,9.4,797
33,Garrett Wilson,WR,7200,13.0,787
73,Zonovan Knight,RB,6300,9.1,747


In [10]:
# highest scoring rosters and frequency
y = top.copy()
y['Roster'] = y.index
y_g = y.groupby('Score', as_index=False).agg({'Roster':'first', 'Salary':'first', 'Projected':'first', 'Hist_Std':'first', 'K':min, 'Score':['first', 'count']})
y_g.sort_values(('Score', 'first'), ascending=False).head(10)

Roster   Salary Projected Hist_Std    K Score      
        first    first     first    first  min first count
90    Roster0  60000.0     115.4    20.91  0.0  8367    32
89    Roster6  60000.0     114.9    21.20  1.0  8359    37
88    Roster2  59800.0     114.1    20.55  1.0  8174    28
87    Roster3  60000.0     115.2    21.46  1.0  8126    15
86   Roster27  60000.0     114.7    21.74  2.0  8118    19
85  Roster534  59900.0     114.5    20.55  2.0  8093    14
84    Roster5  60000.0     115.2    22.43  1.0  8092    27
83  Roster548  59900.0     114.1    21.04  2.0  8061     8
82  Roster529  59900.0     114.4    22.54  2.0  8002    10
81  Roster542  59900.0     114.2    20.11  2.0  7947    11

In [20]:
week.data[week.data['Roster0']==1].iloc[:, :17]

,Nickname,Position,Salary,Injury Indicator,QB,RB,WR,TE,DST,FLEX,Projected,MR_Period,MR_Season,MR_WeekNum,N_Games,Hist_Avg,Hist_Std
0,Justin Jefferson,WR,9300,NaN,0,0,1,0,0,1,21.5,2022-13,2022.0,13.0,16.0,17.26,9.22
27,Tony Pollard,RB,7500,NaN,0,1,0,0,0,1,15.1,2022-13,2022.0,13.0,16.0,13.84,9.79
34,D'Andre Swift,RB,7200,NaN,0,1,0,0,0,1,13.2,2022-13,2022.0,13.0,16.0,12.31,7.12
38,Jared Goff,QB,7100,NaN,1,0,0,0,0,0,18.3,2022-13,2022.0,13.0,16.0,16.54,7.61
47,DJ Moore,WR,6900,NaN,0,0,1,0,0,1,11.1,2022-12,2022.0,12.0,16.0,9.19,6.34
63,Jerry Jeudy,WR,6500,NaN,0,0,1,0,0,1,10.7,2022-13,2022.0,13.0,16.0,7.40,5.44
73,Zonovan Knight,RB,6300,NaN,0,1,0,0,0,1,9.1,2022-13,2022.0,13.0,2.0,13.05,1.77
175,Greg Dulcich,TE,5400,NaN,0,0,0,1,0,1,8.9,2022-13,2022.0,13.0,7.0,7.20,4.31
658,Pittsburgh Steelers,D,3800,NaN,0,0,0,0,1,0,7.5,2022-13,2022.0,13.0,16.0,6.75,7.51


In [25]:
# week.data[week.data['Roster6']==1].iloc[:, :17]

In [22]:
week.data[week.data['Roster2']==1].iloc[:, :17]

,Nickname,Position,Salary,Injury Indicator,QB,RB,WR,TE,DST,FLEX,Projected,MR_Period,MR_Season,MR_WeekNum,N_Games,Hist_Avg,Hist_Std
0,Justin Jefferson,WR,9300,NaN,0,0,1,0,0,1,21.5,2022-13,2022.0,13.0,16.0,17.26,9.22
33,Garrett Wilson,WR,7200,NaN,0,0,1,0,0,1,13.0,2022-13,2022.0,13.0,12.0,10.99,8.59
34,D'Andre Swift,RB,7200,NaN,0,1,0,0,0,1,13.2,2022-13,2022.0,13.0,16.0,12.31,7.12
38,Jared Goff,QB,7100,NaN,1,0,0,0,0,0,18.3,2022-13,2022.0,13.0,16.0,16.54,7.61
41,DeVonta Smith,WR,7000,NaN,0,0,1,0,0,1,11.9,2022-13,2022.0,13.0,16.0,9.88,6.86
63,Jerry Jeudy,WR,6500,NaN,0,0,1,0,0,1,10.7,2022-13,2022.0,13.0,16.0,7.40,5.44
73,Zonovan Knight,RB,6300,NaN,0,1,0,0,0,1,9.1,2022-13,2022.0,13.0,2.0,13.05,1.77
175,Greg Dulcich,TE,5400,NaN,0,0,0,1,0,1,8.9,2022-13,2022.0,13.0,7.0,7.20,4.31
658,Pittsburgh Steelers,D,3800,NaN,0,0,0,0,1,0,7.5,2022-13,2022.0,13.0,16.0,6.75,7.51


## Save Game

In [23]:
# name with 'start em'
week.export_game(file_name = 'start_em_tes')

True

## FanDuel Tournament Export File

In [15]:
play = y_g.sort_values(('Score', 'first'), ascending=False).head(16)

rosters = play[('Roster', 'first')].to_list()

play

Roster   Salary Projected Hist_Std    K Score      
         first    first     first    first  min first count
90     Roster0  60000.0     115.4    20.91  0.0  8367    32
89     Roster6  60000.0     114.9    21.20  1.0  8359    37
88     Roster2  59800.0     114.1    20.55  1.0  8174    28
87     Roster3  60000.0     115.2    21.46  1.0  8126    15
86    Roster27  60000.0     114.7    21.74  2.0  8118    19
85   Roster534  59900.0     114.5    20.55  2.0  8093    14
84     Roster5  60000.0     115.2    22.43  1.0  8092    27
83   Roster548  59900.0     114.1    21.04  2.0  8061     8
82   Roster529  59900.0     114.4    22.54  2.0  8002    10
81   Roster542  59900.0     114.2    20.11  2.0  7947    11
80     Roster8  60000.0     115.1    21.66  1.0  7928    28
79    Roster22  59800.0     113.6    22.12  2.0  7892    24
78    Roster29  59900.0     114.3    21.10  2.0  7852    22
77  Roster1114  59900.0     114.3    21.77  3.0  7838     5
76     Roster4  60000.0     114.4    22.42  1.0  7805    38
75     Roster7  60000.0     114.8    21.69  1.0  7799    24

In [26]:
# rename with start em
csv = week.FanDuelCSVExport(rosters, file_name = 'start_em_tes.csv')

Saving file Lib/FanDuel CSV Imports/start_em_tes.csv


In [27]:
week.data[week.data['Roster2667']==1]

,Nickname,Position,Salary,Injury Indicator,QB,RB,WR,TE,DST,FLEX,...,Roster5102,Roster5103,Roster5104,Roster5105,Roster5106,Roster5107,Roster5108,Roster5109,Roster5110,Top Appearances
0,Justin Jefferson,WR,9300,NaN,0,0,1,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,1023
27,Tony Pollard,RB,7500,NaN,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1278
34,D'Andre Swift,RB,7200,NaN,0,1,0,0,0,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1028
35,Trevor Lawrence,QB,7200,Q,1,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158
63,Jerry Jeudy,WR,6500,NaN,0,0,1,0,0,1,...,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1103
67,Adam Thielen,WR,6400,NaN,0,0,1,0,0,1,...,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,797
73,Zonovan Knight,RB,6300,NaN,0,1,0,0,0,1,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,747
149,Pat Freiermuth,TE,5800,NaN,0,0,0,1,0,1,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,279
658,Pittsburgh Steelers,D,3800,NaN,0,0,0,0,1,0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,879
